In [4]:
import tensorflow as tf
# Load the .pb file
converter = tf.lite.TFLiteConverter.from_saved_model("models/mymodel/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model")

2025-09-05 17:28:39.772474: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-09-05 17:28:39.772885: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-05 17:28:39.772958: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-05 17:28:39.773186: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-05 17:28:39.773472: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
tflite_model = converter.convert()
with open("final.tflite", "wb") as f:
   f.write(tflite_model)

2025-09-05 17:28:58.708590: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-09-05 17:28:58.708679: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-09-05 17:28:58.714411: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/mymodel/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model
2025-09-05 17:28:58.781709: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-09-05 17:28:58.781742: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: models/mymodel/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model
2025-09-05 17:28:58.987902: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-09-05 17:28:59.071189: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-09-05 17:28:59.987264: I tensorflow/cc/saved_model

In [10]:
import tensorflow as tf
import numpy as np

# Paths
saved_model_dir = "models/mymodel/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model"
quantized_model_path = "final_int8.tflite"

# Load SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Optimization for full integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Set input shape
input_shape = [1, 320, 320, 3]  # adjust according to your model

# Representative dataset for calibration
def representative_dataset():
    for _ in range(100):
        # Generate random input in [0, 255] and type uint8
        data = (np.random.rand(*input_shape) * 255).astype(np.uint8)
        yield [data]

converter.representative_dataset = representative_dataset

# Force full INT8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Input/output must be INT8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert and save
tflite_quant_model = converter.convert()
with open(quantized_model_path, "wb") as f:
    f.write(tflite_quant_model)

print(f"INT8 quantized model saved as {quantized_model_path}")


2025-09-05 17:40:18.059356: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-09-05 17:40:18.059469: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-09-05 17:40:18.059731: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/mymodel/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model
2025-09-05 17:40:18.101422: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-09-05 17:40:18.101442: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: models/mymodel/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model
2025-09-05 17:40:18.325348: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-09-05 17:40:19.583116: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: models/mymodel/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_mo

INT8 quantized model saved as final_int8.tflite
